<H2> Preparando o ambiente</H2>

In [1]:
## Instalar pacotes do requests
!pip install -q requests

## Instalar Watermark
!pip install -q watermark

## Instalar o psycopg2-binary -- para suprir dependência da lib sqlalchemy
!pip install psycopg2-binary -q